# PyUSB test
https://www.usb.org/  
https://github.com/pyusb/pyusb  
https://github.com/pyusb/pyusb/blob/master/docs/tutorial.rst  
https://c5techblog.wordpress.com/2016/09/05/tutorial-setup-pyusb-under-windows/  
https://www.beyondlogic.org/usbnutshell/  
https://wwssllabcd.github.io/blog/2012/11/28/usb-emulation/  

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

sudo nano 70-usb.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout", MODE="666" 
SUBSYSTEM=="usb", ATTRS{idVendor}=="0403", GROUP="dialout", MODE="666" 

then reloaded udev rules:

sudo udevadm control --reload-rules  
sudo udevadm trigger

```

In [1]:
%pylab inline  

import usb

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'codes')))

from bridges.ch341A import *
from pprint import pprint

### 掃描 USB 裝置

In [3]:
devs = usb.core.find(find_all = True)
pprint(list(devs))

[<DEVICE ID 0403:6010 on Bus 000 Address 001>,
 <DEVICE ID 046d:c52b on Bus 000 Address 002>,
 <DEVICE ID 04f2:b57e on Bus 000 Address 003>,
 <DEVICE ID 8087:0a2a on Bus 000 Address 004>,
 <DEVICE ID 046d:c52b on Bus 000 Address 005>,
 <DEVICE ID 046d:c52b on Bus 000 Address 006>,
 <DEVICE ID 046d:c52b on Bus 000 Address 007>,
 <DEVICE ID 04f2:b57e on Bus 000 Address 008>,
 <DEVICE ID 0403:6010 on Bus 000 Address 009>,
 <DEVICE ID 0403:6010 on Bus 000 Address 010>]


In [4]:
def find_device_of_class(class_id):
    devices = usb.core.find(find_all = True, bDeviceClass = class_id)
    return (class_id, len(list(devices)))

usb_devices = []
for i in range(256):
    class_id, count = find_device_of_class(i)
    if count > 0:
        devices = usb.core.find(find_all = True, bDeviceClass = class_id)
        usb_devices.append(dict(class_id = class_id, devices = list(devices)))

pprint(usb_devices)

[{'class_id': 0,
  'devices': [<DEVICE ID 152d:0578 on Bus 003 Address 002>,
              <DEVICE ID 0403:6014 on Bus 002 Address 003>,
              <DEVICE ID 046d:c52b on Bus 002 Address 002>]},
 {'class_id': 9,
  'devices': [<DEVICE ID 8087:8001 on Bus 001 Address 002>,
              <DEVICE ID 1d6b:0002 on Bus 001 Address 001>,
              <DEVICE ID 1d6b:0003 on Bus 003 Address 001>,
              <DEVICE ID 1d6b:0002 on Bus 002 Address 001>]},
 {'class_id': 224, 'devices': [<DEVICE ID 0cf3:e300 on Bus 002 Address 006>]},
 {'class_id': 239, 'devices': [<DEVICE ID 13d3:5727 on Bus 002 Address 005>]},
 {'class_id': 255, 'devices': [<DEVICE ID 0bda:0129 on Bus 002 Address 004>]}]


### device

In [4]:
# find our device
dev = usb.core.find(idVendor=0x1a86)  # CH341A
dev = usb.core.find(idVendor=0x0403, idProduct=0x6014)  # FT232H
dev = usb.core.find(idVendor=0x0403, idProduct=0x6010)  # FT2232H

# dev = usb.core.find(idVendor=0x046d, idProduct=0xc52b)  # HID
# dev = usb.core.find(idVendor=0x04f2, idProduct=0xb57e)  # EasyCamera
# dev = usb.core.find(idVendor=0x04f2, idProduct=0xb57e, address = 0x006 )  # EasyCamera


# was it found?
if dev is None:
    raise ValueError('Device not found')
    
usb.util.dispose_resources(dev)
print(dev)

DEVICE ID 0403:6010 on Bus 000 Address 001 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x0403
 idProduct              : 0x6010
 bcdDevice              :  0x700 Device 7.0
 iManufacturer          :    0x1 FTDI
 iProduct               :    0x2 Dual RS232-HS
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 500 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x37 (55 bytes)
   bNumInterfaces       :    0x2
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0x80 Bus Powered
   bMaxPower            :   0xfa (500 mA)
    

In [5]:
# find our device
dev = usb.core.find(idVendor=0x1a86)  # CH341A
dev = usb.core.find(idVendor=0x0403, idProduct=0x6014)  # FT232H
# dev = usb.core.find(idVendor=0x0403, idProduct=0x6010)  # FT2232H

# dev = usb.core.find(idVendor=0x046d, idProduct=0xc52b)  # HID
# dev = usb.core.find(idVendor=0x04f2, idProduct=0xb57e)  # EasyCamera
# dev = usb.core.find(idVendor=0x04f2, idProduct=0xb57e, address = 0x006 )  # EasyCamera


# was it found?
if dev is None:
    raise ValueError('Device not found')
    
usb.util.dispose_resources(dev)
print(dev)

DEVICE ID 0403:6014 on Bus 000 Address 011 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x0403
 idProduct              : 0x6014
 bcdDevice              :  0x900 Device 9.0
 iManufacturer          :    0x1 FTDI
 iProduct               :    0x2 Single RS232-HS
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 500 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x20 (32 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0x80 Bus Powered
   bMaxPower            :   0xfa (500 mA)
  

In [6]:
dev._str()

'DEVICE ID 0403:6014 on Bus 002 Address 003'

### configuration

In [19]:
dev = usb.core.find(idVendor=0x0403, idProduct=0x6014)  # FT232H
# print(dev)

# # set the active configuration. With no arguments, the first
# # configuration will be the active one
dev.set_configuration()

# get an endpoint instance
cfg = dev.get_active_configuration()
print(cfg)

  CONFIGURATION 1: 500 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x20 (32 bytes)
   bNumInterfaces       :    0x1
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0x80 Bus Powered
   bMaxPower            :   0xfa (500 mA)
    INTERFACE 0: Vendor Specific ===========================
     bLength            :    0x9 (9 bytes)
     bDescriptorType    :    0x4 Interface
     bInterfaceNumber   :    0x0
     bAlternateSetting  :    0x0
     bNumEndpoints      :    0x2
     bInterfaceClass    :   0xff Vendor Specific
     bInterfaceSubClass :   0xff
     bInterfaceProtocol :   0xff
     iInterface         :    0x2 Single RS232-HS
      ENDPOINT 0x81: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x81 IN
       bmAttributes     : 

### interface

In [20]:
intf = cfg[(0,0)]
print(intf)

    INTERFACE 0: Vendor Specific ===========================
     bLength            :    0x9 (9 bytes)
     bDescriptorType    :    0x4 Interface
     bInterfaceNumber   :    0x0
     bAlternateSetting  :    0x0
     bNumEndpoints      :    0x2
     bInterfaceClass    :   0xff Vendor Specific
     bInterfaceSubClass :   0xff
     bInterfaceProtocol :   0xff
     iInterface         :    0x2 Single RS232-HS
      ENDPOINT 0x81: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x81 IN
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x200 (512 bytes)
       bInterval        :    0x0
      ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x200 (512 bytes)
       bInterval        : 

### endpoint

In [21]:
ep = usb.util.find_descriptor(intf,  # match the first OUT endpoint
                              custom_match = lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == ENDPOINT.DIRECTION.OUT)

print(ep)

      ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x200 (512 bytes)
       bInterval        :    0x0


### Buld read/write to detect I2C address (using CH341)

In [22]:
# detect I2C address
addr = 120
cmd = [CH341.PORT.I2C, I2C.STA, I2C.OUT, addr, I2C.STO, I2C.END]
cmd

AttributeError: type object 'I2C' has no attribute 'STA'

In [23]:
buff = usb._interop.as_array(cmd)
buff

NameError: name 'cmd' is not defined

In [ ]:
count = dev.write(CH341.ENDPOINTS.BULK_OUT, cmd)
count

In [ ]:
value = dev.read(CH341.ENDPOINTS.BULK_IN, I2C.MAX)
value

In [ ]:
detected = not (value[0] & 0x80)
detected

In [ ]:
# dev = usb.core.find(idVendor=0xfffe, idProduct=0x0001)
# if dev is None:
#     raise ValueError('Our device is not connected')

In [ ]:
# # actually this is not the whole history, keep reading
# if usb.core.find(bDeviceClass = 7) is None:
#     raise ValueError('No printer found')

In [ ]:
# USBdevice.scan(full = True)

### 條件式 掃描 USB 裝置

In [ ]:
import usb.core
import usb.util
import sys

class find_class(object):
    def __init__(self, class_):
        self._class = class_
    def __call__(self, device):
        # first, let's check the device
        if device.bDeviceClass == self._class:
            return True
        # ok, transverse all devices to find an
        # interface that matches our class
        for cfg in device:
            # find_descriptor: what's it?
            intf = usb.util.find_descriptor(cfg,
                                            bInterfaceClass = self._class)
            if intf is not None:
                return True

        return False

devices = usb.core.find(find_all=1, custom_match=find_class(255))
list(devices)

### configuration

In [ ]:
for cfg in dev:
    print(cfg.bConfigurationValue)

In [ ]:
# access the second configuration
dev.set_configuration()

In [ ]:
cfg = dev[0]
cfg.__dict__

### interface

In [ ]:
# access the first interface
intf = cfg[(0,0)]
intf.__dict__

In [ ]:
print(intf)

### endpoint

In [ ]:
# third endpoint
ep = intf[0] 
ep.__dict__

In [ ]:
print(ep)

In [ ]:
ep._str()

In [ ]:
# >>> msg = 'test'
# >>> assert dev.ctrl_transfer(0x40, CTRL_LOOPBACK_WRITE, 0, 0, msg) == len(msg)
# >>> ret = dev.ctrl_transfer(0xC0, CTRL_LOOPBACK_READ, 0, 0, len(msg))
# >>> sret = ''.join([chr(x) for x in ret])
# >>> assert sret == msg

### Release resources

In [ ]:
usb.util.release_interface(dev, intf)

In [ ]:
usb.util.dispose_resources(dev)

In [ ]:
# dev.reset()

In [ ]:
import usb.util

alt = usb.util.find_descriptor(cfg, find_all=True, bInterfaceNumber = 0x00, bAlternateSetting = 0x00)
list(alt)

### set_configuration

In [ ]:
dev.set_configuration(1)

In [ ]:
# or
dev.set_configuration() # we assume the configuration 5 is the first one

In [ ]:
dev.get_active_configuration()

In [ ]:
# or
cfg = usb.util.find_descriptor(dev, bConfigurationValue=1)
cfg.set()

In [ ]:
# or
cfg = usb.util.find_descriptor(dev, bConfigurationValue = 0x01)
dev.set_configuration(cfg)

### set alternative interface

In [ ]:
dev.set_interface_altsetting(interface = 0, alternate_setting = 0)

In [ ]:
usb.util.dispose_resources(dev)